# Date Grammar Testing

Testing date parsing with optional years and smart year inference.
Supports formats like: `Jan 2`, `Jan 2 26`, `1/15`, `1/15/2026`

In [1]:
from rehearsal_scheduler.grammar import validate_token

In [2]:
from rehearsal_scheduler.grammar import constraint_parser
constraint_text = "m"
parser = constraint_parser(debug=False)
result = parser.parse(constraint_text)
print(f"\nres = {result}")


res = (DayOfWeekConstraint(day_of_week='monday'),)


In [3]:
bad_tokens = [
    "w 0am",              # Invalid 12-hour format
    "th 5-2pm",           # Start time is after end time
    "th 11:30am-1100",    # Start time is after end time
    "th after 10:61 am",  # Start time is after end time
    "m 1500-1300",        # military time, Start time is after end time
    "12/15/26",
    "jan 2",
    "jan 15",
    "Feb 29 2023",
    "13/15/26",
    "XYZ 15 26",
    "m after 25",    # Invalid time
    "tues 13pm-2pm",

]   

for token in bad_tokens:
    result, error_message = validate_token(token)
    if result is None:
        print(error_message)
    else:
        print(result)
    print()

w 0am
   ^
Expected one of {'HOUR', 'MILITARY_TIME', 'AFTER', 'UNTIL', 'BEFORE', 'COMMA'}

th 5-2pm: Start time 17:00:00 must be before end time 14:00:00.

th 11:30am-1100: Start time 11:30:00 must be before end time 11:00:00.

th after 10:61 am
            ^
Expected: {'MINUTE'}

m 1500-1300: Start time 15:00:00 must be before end time 13:00:00.

(DateConstraint(date=2026-12-15),)

jan 2
    ^
Expected: {'YEAR'}

jan 15
    ^
Expected: {'YEAR'}

Feb 29 2023: Invalid date: day is out of range for month

13/15/26
 ^
Expected: {'SLASH'}

XYZ 15 26
 ^
Expected one of {'TUESDAY', 'MONTH_NUM', 'MONDAY', 'MONTH_TEXT', 'WEDNESDAY', 'SATURDAY', 'SUNDAY', 'FRIDAY', 'THURSDAY'}

m after 25
         ^
Expected: {'COLON', 'AM_PM', 'MINUS', 'COMMA'}

tues 13pm-2pm
      ^
Expected: {'COLON', 'AM_PM', 'MINUS', 'COMMA'}



In [12]:
from lark.exceptions import UnexpectedInput, UnexpectedCharacters, UnexpectedToken
from rehearsal_scheduler.grammar import constraint_parser, SemanticValidationError
parser = constraint_parser(debug=False)

for text in bad_tokens:
    try:
        result = parser.parse(text)
    except UnexpectedCharacters:
        pass
    except UnexpectedToken:
        pass
    except SemanticValidationError:
        pass
    except ValueError:
        pass
print(f"\nres = {result}")


res = (DateConstraint(date=2026-12-15),)


In [4]:
# Test edge cases and errors
print("--- Testing Invalid Dates ---\n")

invalid_dates = [
    "FEB 29 2023",  # Not a leap year
    "13/15",         # Invalid month
    "11/31",         # November only has 30 days
    "APR 31",        # April only has 30 days
    "m after 25",    # Invalid time
]

for text in invalid_dates:
    try:
        result = parser.parse(text)
        print(f'❌ "{text}" unexpectedly succeeded: {result}')
    except Exception as e:
        print(f'✅ "{text}" correctly rejected: {type(e).__name__}')

# parser = Lark(DATE_GRAMMAR, parser="lalr", transformer=DateValidator())

--- Testing Invalid Dates ---

✅ "FEB 29 2023" correctly rejected: ValueError
✅ "13/15" correctly rejected: UnexpectedToken
✅ "11/31" correctly rejected: UnexpectedToken
✅ "APR 31" correctly rejected: UnexpectedToken
✅ "m after 25" correctly rejected: UnexpectedToken


In [5]:
# Test single dates
print("Single dates:")
print(parser.parse("Jan 2 26"))           # -> date(2026, 1, 2)
print(parser.parse("1/15/25"))            # -> date(2025, 1, 15)

# Test date ranges
print("\nDate ranges:")
print(parser.parse("Jan 1 26 - Jan 5 26"))       # -> (date(2026, 1, 1), date(2026, 1, 5))
print(parser.parse("12/20/25 - 12/25/25"))       # -> (date(2025, 12, 20), date(2025, 12, 25))
print(parser.parse("Feb 1 26-Feb 10 26"))        # -> (date(2026, 2, 1), date(2026, 2, 10))

# Test invalid range
try:
    print(parser.parse("Jan 10 26 - Jan 5 26"))  # end before start
except ValueError as e:
    print(f"Error: {e}")

Single dates:
(DateConstraint(date=2026-01-02),)
(DateConstraint(date=2025-01-15),)

Date ranges:
(DateRangeConstraint(start=2026-01-01, end=2026-01-05),)
(DateRangeConstraint(start=2025-12-20, end=2025-12-25),)
(DateRangeConstraint(start=2026-02-01, end=2026-02-10),)
Error: Invalid range: end date 2026-01-05 is before start date 2026-01-10


In [6]:
# Test parsing errors with helpful messages
from lark.exceptions import UnexpectedInput, UnexpectedToken

test_input = "Jan 2"  # Missing year

try:
    result = parser.parse(test_input)
    print(f"Unexpectedly succeeded: {result}")
except UnexpectedInput as e:
    print(f"Input: '{test_input}'")
    print(f"Error at position {e.pos_in_stream}: {e.token}")
    print(f"Expected: {e.expected}")
    print(f"\nFull error:\n{e}")

Input: 'Jan 2'
Error at position 4: 
Expected: {'YEAR'}

Full error:
Unexpected token Token('$END', '') at line 1, column 5.
Expected one of: 
	* YEAR

